In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet18")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    val_loss_TL = []
    val_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    #資料抓入
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, val = epg.getTrainDataset(fold, sound[TL], num_class)
        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
        val_loss = torch.stack(val_loss,dim=-1).cpu()
        val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        val_loss_TL.append(val_loss)
        val_accuracy_TL.append(val_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    val_loss_array.append(val_loss_TL)
    val_accuracy_array.append(val_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 0.9174089431762695 Train_acc: 57.437500% Val_Loss: 1.8436366319656372 Val_accuracy: 61.500000%
Test accuracy of the model: 56.400000%
Test accuracy of the model: 55.666667%
Test accuracy of the model: 61.000000%
Train Epoch: 2/30 Train_Loss: 0.6085714101791382 Train_acc: 79.500000% Val_Loss: 0.6366174817085266 Val_accuracy: 62.500000%
Test accuracy of the model: 95.600000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 3/30 Train_Loss: 0.4020983874797821 Train_acc: 86.375000% Val_Loss: 1.7703630924224854 Val_accuracy: 65.250000%
Test accuracy of the model: 86.200000%
Test accuracy of the model: 84.666667%
Test accuracy of the model: 82.000000%
Train Epoch: 4/30 Train_Loss: 0.

Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 3/30 Train_Loss: 0.17105711996555328 Train_acc: 96.770836% Val_Loss: 1.2853577136993408 Val_accuracy: 73.333336%
Test accuracy of the model: 56.200000%
Test accuracy of the model: 55.666667%
Test accuracy of the model: 54.000000%
Train Epoch: 4/30 Train_Loss: 0.027288179844617844 Train_acc: 97.291672% Val_Loss: 1.2058442831039429 Val_accuracy: 76.250008%
Test accuracy of the model: 93.800000%
Test accuracy of the model: 93.000000%
Test accuracy of the model: 92.000000%
Train Epoch: 5/30 Train_Loss: 0.10725825279951096 Train_acc: 97.187508% Val_Loss: 0.83074951171875 Val_accuracy: 74.166672%
Test accuracy of the model: 93.400000%
Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.11374031007289886 Train_acc: 97.812508% Val_Loss: 0.6319798827171326 Val_accuracy: 77.500008%
Test accuracy of the model: 96.800000%
Test accuracy of the model

Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.11449930816888809 Train_acc: 99.062500% Val_Loss: 1.4600708484649658 Val_accuracy: 80.000000%
Test accuracy of the model: 97.800000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 7/30 Train_Loss: 0.1336556226015091 Train_acc: 99.062500% Val_Loss: 0.8988989591598511 Val_accuracy: 80.000000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 8/30 Train_Loss: 0.01795162633061409 Train_acc: 99.375000% Val_Loss: 0.7545696496963501 Val_accuracy: 83.750000%
Test accuracy of the model: 92.800000%
Test accuracy of the model: 94.000000%
Test accuracy of the model: 95.000000%
Train Epoch: 9/30 Train_Loss: 0.013804679736495018 Train_acc: 99.687500% Val_Loss: 0.7325323820114136 Val_accuracy: 83.750000%
Test accuracy of the mod

Train Epoch: 8/30 Train_Loss: 0.15078286826610565 Train_acc: 92.187500% Val_Loss: 0.9015287160873413 Val_accuracy: 77.750000%
Test accuracy of the model: 94.400000%
Test accuracy of the model: 94.333333%
Test accuracy of the model: 96.000000%
Train Epoch: 9/30 Train_Loss: 0.1741633415222168 Train_acc: 91.750000% Val_Loss: 0.814105749130249 Val_accuracy: 78.750000%
Test accuracy of the model: 91.600000%
Test accuracy of the model: 95.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 10/30 Train_Loss: 0.38351917266845703 Train_acc: 92.125000% Val_Loss: 0.8298016786575317 Val_accuracy: 75.500000%
Test accuracy of the model: 90.200000%
Test accuracy of the model: 90.000000%
Test accuracy of the model: 90.000000%
Train Epoch: 11/30 Train_Loss: 0.12389639765024185 Train_acc: 93.562500% Val_Loss: 0.8448081016540527 Val_accuracy: 81.250000%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 97.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 12/30 Train_Los

Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 11/30 Train_Loss: 0.04951360821723938 Train_acc: 98.750008% Val_Loss: 0.30508145689964294 Val_accuracy: 90.416672%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 12/30 Train_Loss: 0.06595059484243393 Train_acc: 98.750008% Val_Loss: 0.6415669322013855 Val_accuracy: 89.166672%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 13/30 Train_Loss: 0.14952169358730316 Train_acc: 98.750008% Val_Loss: 0.5817976593971252 Val_accuracy: 88.750008%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 14/30 Train_Loss: 0.056658852845430374 Train_acc: 98.854172% Val_Loss: 0.3754137456417084 Val_accuracy: 89.583336%
Test accuracy of the model: 95.800000%
Test accuracy of 

...

Test accuracy of the model: 93.200000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 24/30 Train_Loss: 0.08330853283405304 Train_acc: 94.562500% Val_Loss: 0.7097854614257812 Val_accuracy: 79.000000%
Test accuracy of the model: 93.600000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 25/30 Train_Loss: 0.05675850436091423 Train_acc: 94.562500% Val_Loss: 0.3654695153236389 Val_accuracy: 80.000000%
Test accuracy of the model: 93.800000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 26/30 Train_Loss: 0.053083647042512894 Train_acc: 94.875000% Val_Loss: 1.0544098615646362 Val_accuracy: 79.000000%
Test accuracy of the model: 93.600000%
Test accuracy of the model: 96.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 27/30 Train_Loss: 0.06998135894536972 Train_acc: 95.062500% Val_Loss: 0.20759868621826172 Val_accuracy: 79.750000%
Test accuracy of the m

Test accuracy of the model: 94.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 27/30 Train_Loss: 0.0043925512582063675 Train_acc: 98.854172% Val_Loss: 0.33205297589302063 Val_accuracy: 88.333336%
Test accuracy of the model: 94.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 28/30 Train_Loss: 0.03571019694209099 Train_acc: 98.750008% Val_Loss: 0.6491500735282898 Val_accuracy: 87.916672%
Test accuracy of the model: 94.600000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 29/30 Train_Loss: 0.03111915849149227 Train_acc: 98.854172% Val_Loss: 0.33922097086906433 Val_accuracy: 87.916672%
Test accuracy of the model: 94.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 30/30 Train_Loss: 0.010806707665324211 Train_acc: 98.854172% Val_Loss: 0.5136215090751648 Val_accuracy: 90.416672%
Test accuracy o

Test accuracy of the model: 100.000000%
Train Epoch: 29/30 Train_Loss: 0.0008493413333781064 Train_acc: 100.000000% Val_Loss: 0.21659532189369202 Val_accuracy: 92.500000%
Test accuracy of the model: 94.800000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 30/30 Train_Loss: 0.00047824662760831416 Train_acc: 100.000000% Val_Loss: 0.3411675989627838 Val_accuracy: 91.250000%
Test accuracy of the model: 95.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  5
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 0.872961699962616 Train_acc: 55.187500% Val_Loss: 1.2560292482376099 Val_accuracy: 62.250000%
Test accuracy of the model: 75.200000%
Test accuracy of the model: 85.666667%
Test accuracy of t

Train Epoch: 1/30 Train_Loss: 0.20037271082401276 Train_acc: 92.812508% Val_Loss: 0.6288780570030212 Val_accuracy: 83.333336%
Test accuracy of the model: 90.600000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 2/30 Train_Loss: 0.05972227081656456 Train_acc: 96.354172% Val_Loss: 0.6337809562683105 Val_accuracy: 79.583336%
Test accuracy of the model: 93.600000%
Test accuracy of the model: 100.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 3/30 Train_Loss: 0.10410448163747787 Train_acc: 96.875008% Val_Loss: 1.0765390396118164 Val_accuracy: 82.083336%
Test accuracy of the model: 94.600000%
Test accuracy of the model: 96.666667%
Test accuracy of the model: 98.000000%
Train Epoch: 4/30 Train_Loss: 0.10936024785041809 Train_acc: 98.020836% Val_Loss: 0.6620739102363586 Val_accuracy: 82.083336%
Test accuracy of the model: 91.800000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 5/30 Train_

Test accuracy of the model: 94.400000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 4/30 Train_Loss: 0.04560649394989014 Train_acc: 99.375000% Val_Loss: 0.6929931640625 Val_accuracy: 82.500000%
Test accuracy of the model: 95.400000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 5/30 Train_Loss: 0.0497661828994751 Train_acc: 99.687500% Val_Loss: 0.8030598759651184 Val_accuracy: 82.500000%
Test accuracy of the model: 94.800000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.0041223024018108845 Train_acc: 99.687500% Val_Loss: 1.1072982549667358 Val_accuracy: 72.500000%
Test accuracy of the model: 96.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 7/30 Train_Loss: 0.036288075149059296 Train_acc: 100.000000% Val_Loss: 1.730782151222229 Val_accuracy: 71.250000%
Test accuracy of the model

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    val_loss_last = []
    val_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        val_loss_last.append(val_loss_array[fold][TL][num_epochs-1])
        val_accuracy_last.append(val_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], val_loss_array[fold][TL][num_epochs-1], val_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()